In [ ]:
# pip install nbformat --upgrade


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import xgboost as xgb
import plotly.graph_objects as go
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc, accuracy_score, confusion_matrix, precision_recall_fscore_support, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.utils import resample
warnings.filterwarnings('ignore')


In [ ]:
# Load data
data = pd.read_csv('augmented_file_50x.csv')

In [ ]:
data['label'].value_counts()

In [ ]:

# Features list
features = ['N', 'P', 'K', 'temperature', 'ph', 'rainfall']

# Compute skewness and kurtosis
skewness = data[features].skew()
kurtosis = data[features].kurt()

# Create a new DataFrame to compile all statistics
stats_df = pd.DataFrame({
    'Mean': data[features].mean(),
    'Std Dev': data[features].std(),
    'Min': data[features].min(),
    'Max': data[features].max(),
    'Skewness': skewness,
    'Kurtosis': kurtosis
})

# Round the statistics to desired decimal places
stats_df = stats_df.round(2)

# Replace -0.00 with 0.00 for cleanliness
stats_df = stats_df.replace(-0.00, 0.00)

print(stats_df)


In [ ]:
# Visualizing the data distribution between N, P, and K

# Set the size for the overall figure
plt.figure(figsize=(18, 6))

# Plotting the histogram and KDE for N
plt.subplot(1, 3, 1)
sns.histplot(data['N'], color="red", bins=15, kde=True, alpha=0.2)
plt.title('Distribution of N')

# Plotting the histogram and KDE for P
plt.subplot(1, 3, 2)
sns.histplot(data['P'], color="blue", bins=15, kde=True, alpha=0.2)
plt.title('Distribution of P')

# Plotting the histogram and KDE for K
plt.subplot(1, 3, 3)
sns.histplot(data['K'], color="green", bins=15, kde=True, alpha=0.2)
plt.title('Distribution of K')

# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
# Visualizing the data distribution between temperature, ph and rainfall

plt.figure(figsize=(12, 5))

# Plotting the histogram and KDE for temperature
plt.subplot(1, 3, 1)
sns.histplot(data['temperature'], color="red", bins=15, kde=True, alpha=0.2)
plt.title('Distribution of Temperature')

# Plotting the histogram and KDE for ph
plt.subplot(1, 3, 2)
sns.histplot(data['ph'], color="blue", bins=15, kde=True, alpha=0.2)
plt.title('Distribution of pH')

# Plotting the histogram and KDE for rainfall
plt.subplot(1, 3, 3)
sns.histplot(data['rainfall'], color="green", bins=15, kde=True, alpha=0.2)
plt.title('Distribution of Rainfall')

plt.tight_layout()
plt.show()



In [ ]:
# # Basic pairplot with enhancements
# sns.pairplot(data,
#              hue='label',
#              palette='pastel',       # Use pastel palette for distinction and aesthetics
#              height=2.5,             # Increase plot size
#              aspect=1,               # Aspect ratio
#              diag_kind='kde',        # KDE plots on the diagonal
#              plot_kws={'alpha': 0.7, 's': 80, 'edgecolor': 'k'},  # Opacity, size, and edge color of markers
#              diag_kws={'shade': True}   # Shade the KDE plots
#              )

# # Show plot
# plt.show()


In [ ]:
# Compute the correlation matrix
corr = data[features].corr()

# Plot the heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1, square=True, linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()


In [ ]:

# Split data into features and labels
X = data.drop('label', axis=1)
y = data['label']

# Lists to collect split data
X_train_list = []
X_val_list = []
X_test_list = []
y_train_list = []
y_val_list = []
y_test_list = []

# Split each label's data separately
labels = y.unique()
for label in labels:
    X_label = X[y == label]
    y_label = y[y == label]
    
    # First, separate out the training data (70%)
    X_train_label, X_temp_label, y_train_label, y_temp_label = train_test_split(
        X_label, y_label, test_size=0.30, random_state=0)
    
    # Now, split the remaining data into validation and test sets (20% test, 10% validation)
    X_test_label, X_val_label, y_test_label, y_val_label = train_test_split(
        X_temp_label, y_temp_label, test_size=0.3333, random_state=0)
    
    X_train_list.append(X_train_label)
    X_val_list.append(X_val_label)
    X_test_list.append(X_test_label)
    y_train_list.append(y_train_label)
    y_val_list.append(y_val_label)
    y_test_list.append(y_test_label)

# Concatenate splits
X_train = pd.concat(X_train_list, axis=0)
y_train = pd.concat(y_train_list, axis=0)
X_val = pd.concat(X_val_list, axis=0)
y_val = pd.concat(y_val_list, axis=0)
X_test = pd.concat(X_test_list, axis=0)
y_test = pd.concat(y_test_list, axis=0)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Testing set size:", len(X_test))


In [ ]:
# Using 'auto' strategy for RandomUnderSampler
under = RandomUnderSampler(sampling_strategy='auto')
X_resampled, y_resampled = under.fit_resample(X_train, y_train)

# Train the Random Forest on the resampled data
clf_rf = RandomForestClassifier(n_estimators=100, max_depth=6, max_features=4, random_state=0)
clf_rf.fit(X_resampled, y_resampled)

# Perform 10-fold cross-validation on the resampled data
cv_scores = cross_val_score(clf_rf, X_resampled, y_resampled, cv=10, scoring='accuracy')

# Print out the scores for each fold and average accuracy
print("Cross-validated accuracy scores for each fold:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
print("Standard Deviation of CV Accuracy:", cv_scores.std())

# Calibrate the classifier on the validation set
calibrated_rf = CalibratedClassifierCV(clf_rf, method='sigmoid', cv='prefit')
calibrated_rf.fit(X_val, y_val)

# Predict on the validation set using the calibrated RF
y_pred_val = calibrated_rf.predict(X_val)
accuracy_score_val = accuracy_score(y_val, y_pred_val)
print("Calibrated Random Forest Validation accuracy:", accuracy_score_val)

# Predict on the test set using the calibrated RF
y_pred_calibrated = calibrated_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_calibrated)
print("Calibrated Random Forest Test accuracy:", accuracy_rf)
print("Calibrated Random Forest Test Classification Report:")
print(classification_report(y_test, y_pred_calibrated))

# Compute metrics for model_metrics dictionary
precision_rf, recall_rf, f1_rf, _ = precision_recall_fscore_support(y_test, y_pred_calibrated, average='weighted')
model_metrics_rf= {
    "Random Forest": {
        "Accuracy": f"{accuracy_rf * 100:.2f}%",
        "Precision": f"{precision_rf * 100:.2f}%",
        "Recall": f"{recall_rf * 100:.2f}%",
        "F1 Score": f"{f1_rf * 100:.2f}%"
    }
}
print("Model Metrics for Random Forest", model_metrics_rf)

# Compute the confusion matrix for calibrated RF
conf_mat_calibrated = confusion_matrix(y_test, y_pred_calibrated)
class_labels = np.unique(y_test)

# Plot using Seaborn for calibrated RF
plt.figure(figsize=(11,7))
sns.heatmap(conf_mat_calibrated, annot=True, fmt="d", cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for Calibrated Random Forest')
plt.show()


In [ ]:
# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val) # scale the validation set
X_test_scaled = scaler.transform(X_test)

# Using 'auto' strategy for RandomUnderSampler
under = RandomUnderSampler(sampling_strategy='auto')

# Apply the undersampling strategy
X_train_resampled, y_train_resampled = under.fit_resample(X_train_scaled, y_train)

# Define the K-nearest neighbors (KNN) classifier with adjusted parameters
clf_knn = KNeighborsClassifier(n_neighbors=5001, weights='distance', algorithm='auto', p=2)

# Fit the KNN model with resampled training data
clf_knn.fit(X_train_resampled, y_train_resampled)

# Perform 10-fold cross-validation on the resampled data
cv_scores_knn = cross_val_score(clf_knn, X_train_resampled, y_train_resampled, cv=10, scoring='accuracy')

# Print out the scores for each fold and average accuracy
print("Cross-validated accuracy scores for each fold (KNN):", cv_scores_knn)
print("Mean CV Accuracy (KNN):", cv_scores_knn.mean())
print("Standard Deviation of CV Accuracy (KNN):", cv_scores_knn.std())

# Predict using the KNN model on the scaled validation set
y_pred_knn_val = clf_knn.predict(X_val_scaled)
accuracy_score_knn_val = accuracy_score(y_val, y_pred_knn_val)
print("K-nearest Neighbors Validation accuracy:", accuracy_score_knn_val)

# Predict using the KNN model on the scaled test set
y_pred_knn = clf_knn.predict(X_test_scaled)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("K-nearest Neighbors Test accuracy:", accuracy_knn)
print("K-nearest Neighbors Test Classification Report:")
print(classification_report(y_test, y_pred_knn))

# Compute metrics for model_metrics dictionary
precision_knn, recall_knn, f1_knn, _ = precision_recall_fscore_support(y_test, y_pred_knn, average='weighted')
model_metrics_knn = {
    "Accuracy": f"{accuracy_knn * 100:.2f}%",
    "Precision": f"{precision_knn * 100:.2f}%",
    "Recall": f"{recall_knn * 100:.2f}%",
    "F1 Score": f"{f1_knn * 100:.2f}%"
}

# Compute the confusion matrix for test set
conf_mat_knn = confusion_matrix(y_test, y_pred_knn)

# Use the previously established method to extract unique class labels
class_labels = np.unique(y_test)

# Plot using Seaborn
plt.figure(figsize=(11,7))
sns.heatmap(conf_mat_knn, annot=True, fmt="d", cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for K-nearest Neighbors on Test Set')
plt.show()


In [ ]:

# Set up the RandomUnderSampler
under = RandomUnderSampler(sampling_strategy='auto')
X_train_under, y_train_under = under.fit_resample(X_train, y_train)

# Set var_smoothing parameter for regularization
var_smoothing_value = 1e-1
clf_nb = GaussianNB(var_smoothing=var_smoothing_value)

# Perform 10-fold cross-validation on undersampled data
cv_scores = cross_val_score(clf_nb, X_train_under, y_train_under, cv=10, scoring='accuracy')

# Print out the scores for each fold and average accuracy
print("Cross-validated accuracy scores for each fold:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
print("Standard Deviation of CV Accuracy:", cv_scores.std())

# Fit the clf_nb with the undersampled training data
clf_nb.fit(X_train_under, y_train_under)

# Calibrate the classifier
calibrated_nb = CalibratedClassifierCV(clf_nb, method='sigmoid', cv='prefit')
calibrated_nb.fit(X_val, y_val)

# Predict for the training set using the calibrated model
y_pred_nb_train = calibrated_nb.predict(X_train)
accuracy_score_nb_train = accuracy_score(y_train, y_pred_nb_train)

# Predict for the validation set using the calibrated model
y_pred_nb_val = calibrated_nb.predict(X_val)
accuracy_score_nb_val = accuracy_score(y_val, y_pred_nb_val)

# Predict for the test set using the calibrated model
y_pred_nb = calibrated_nb.predict(X_test)
accuracy_nb = accuracy_score(y_test, y_pred_knn)
print("Naive Bayes Validation accuracy:", accuracy_score_nb_val)
print("Naive Bayes Test accuracy:", accuracy_nb)
print("Naive Bayes Test Classification Report:")
print(classification_report(y_test, y_pred_nb))

# Compute metrics for model_metrics dictionary
precision_nb, recall_nb, f1_nb, _ = precision_recall_fscore_support(y_test, y_pred_nb, average='weighted')
model_metrics_nb = {
    "Accuracy": f"{accuracy_nb * 100:.2f}%",
    "Precision": f"{precision_nb * 100:.2f}%",
    "Recall": f"{recall_nb * 100:.2f}%",
    "F1 Score": f"{f1_nb * 100:.2f}%"
}
# Compute the confusion matrix for the test set
conf_mat_nb = confusion_matrix(y_test, y_pred_nb)

# Extract unique class labels
class_labels = np.unique(y)

# Plot using Seaborn for the test set
plt.figure(figsize=(11,7))
sns.heatmap(conf_mat_nb, annot=True, fmt="d", cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for Naive Bayes on Test Set')
plt.show()



In [ ]:
# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression with stronger regularization (smaller C value)
clf_lr = LogisticRegression(C=0.001, max_iter=5000, random_state=0)
clf_lr.fit(X_train_scaled, y_train)

# Calibrating the classifier using the previously fitted instance
calibrated_clf = CalibratedClassifierCV(clf_lr, method='sigmoid', cv='prefit')
calibrated_clf.fit(X_val_scaled, y_val)

# Perform 10-fold cross-validation on the training data with original classifier
cv_scores = cross_val_score(clf_lr, X_train_scaled, y_train, cv=10, scoring='accuracy')

# Print out the scores for each fold and average accuracy
print("Cross-validated accuracy scores for each fold:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
print("Standard Deviation of CV Accuracy:", cv_scores.std())

# Predict on the validation set using calibrated classifier
y_pred_lr_val = calibrated_clf.predict(X_val_scaled)
accuracy_score_lr_val = accuracy_score(y_val, y_pred_lr_val)
print("Logistic Regression Validation accuracy:", accuracy_score_lr_val)

# Predict on the test set using calibrated classifier
y_pred_lr = calibrated_clf.predict(X_test_scaled)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

# Print the Logistic Regression Classification Report for the test set
print("Logistic Regression Test accuracy:", accuracy_lr)
print("Logistic Regression Test Classification Report:")
print(classification_report(y_test, y_pred_lr))

# Compute metrics for model_metrics dictionary
precision_lr, recall_lr, f1_lr, _ = precision_recall_fscore_support(y_test, y_pred_lr, average='weighted')
model_metrics_lr = {
    "Accuracy": f"{accuracy_lr * 100:.2f}%",
    "Precision": f"{precision_lr * 100:.2f}%",
    "Recall": f"{recall_lr * 100:.2f}%",
    "F1 Score": f"{f1_lr * 100:.2f}%"
}

# Compute the confusion matrix for the test set
conf_mat_lr = confusion_matrix(y_test, y_pred_lr)

# Extract unique class labels
class_labels = np.unique(y)

# Plot using Seaborn for the test set
plt.figure(figsize=(11,7))
sns.heatmap(conf_mat_lr, annot=True, fmt="d", cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for Logistic Regression on Test Set')
plt.show()


In [ ]:
# Convert string labels to integers
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)
y_test_encoded = le.transform(y_test)

# Define the XGBoost classifier
clf_xgb = xgb.XGBClassifier(
    n_estimators=5,
    max_depth=3,
    min_child_weight=5,
    gamma=0.01,
    learning_rate=0.03,
    objective='multi:softmax',
    random_state=0
)

# Fit the model to the training data
clf_xgb.fit(X_train, y_train_encoded)

# Perform 10-fold cross-validation on the original XGBClassifier
cv_scores = cross_val_score(clf_xgb, X_train, y_train_encoded, cv=10, scoring='accuracy')

# Print out the scores for each fold and average accuracy
print("Cross-validated accuracy scores for each fold:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
print("Standard Deviation of CV Accuracy:", cv_scores.std())

# Calibrate the classifier AFTER the cross-validation
calibrated_xgb = CalibratedClassifierCV(clf_xgb, method='sigmoid', cv='prefit')
calibrated_xgb.fit(X_val, y_val_encoded)

# Predict on the validation set using the calibrated classifier
y_pred_xgb_val = le.inverse_transform(calibrated_xgb.predict(X_val))

# Compute the accuracy for the validation set
accuracy_score_xgb_val = accuracy_score(y_val, y_pred_xgb_val)
print("Calibrated XGBoost Validation accuracy:", accuracy_score_xgb_val)

# Decode the predicted labels to original string format for the test set
y_pred_xgb = le.inverse_transform(calibrated_xgb.predict(X_test))

# Use the decoded predicted labels for calculating accuracy and plotting confusion matrix
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("XGBoost Test accuracy:", accuracy_xgb)
print("XGBoost Test Classification Report:")
print(classification_report(y_test, y_pred_xgb))

# Extract unique class labels from y_train
class_labels = np.unique(y_train)

# Compute metrics for model_metrics dictionary
precision_xgb, recall_xgb, f1_xgb, _ = precision_recall_fscore_support(y_test, y_pred_xgb, average='weighted')
model_metrics_xgb = {
    "Accuracy": f"{accuracy_xgb * 100:.2f}%",
    "Precision": f"{precision_xgb * 100:.2f}%",
    "Recall": f"{recall_xgb * 100:.2f}%",
    "F1 Score": f"{f1_xgb * 100:.2f}%"
}

# Compute the confusion matrix
conf_mat_xgb = confusion_matrix(y_test, y_pred_xgb)

# Plot using Seaborn
plt.figure(figsize=(11,7))
sns.heatmap(conf_mat_xgb, annot=True, fmt="d", cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for Calibrated XGBoost')
plt.show()


In [ ]:


# Compute metrics
metrics = {
    "Model": ["Random Forest", "KNN", "Naive Bayes", "Logistic Regression", "XGBoost"],
    "Accuracy": [accuracy_rf, accuracy_knn, accuracy_nb, accuracy_lr, accuracy_xgb],
    "Precision": [precision_score(y_test, y_pred_calibrated, average='weighted'),
                  precision_score(y_test, y_pred_knn, average='weighted'),
                  precision_score(y_test, y_pred_nb, average='weighted'),
                  precision_score(y_test, y_pred_lr, average='weighted'),
                  precision_score(y_test, y_pred_xgb, average='weighted')],
    "Recall": [recall_score(y_test, y_pred_calibrated, average='weighted'),
               recall_score(y_test, y_pred_knn, average='weighted'),
               recall_score(y_test, y_pred_nb, average='weighted'),
               recall_score(y_test, y_pred_lr, average='weighted'),
               recall_score(y_test, y_pred_xgb, average='weighted')],
    "F1 Score": [f1_score(y_test, y_pred_calibrated, average='weighted'),
                 f1_score(y_test, y_pred_knn, average='weighted'),
                 f1_score(y_test, y_pred_nb, average='weighted'),
                 f1_score(y_test, y_pred_lr, average='weighted'),
                 f1_score(y_test, y_pred_xgb, average='weighted')]
}

# Print in tabular form
print("{:<20} {:<10} {:<10} {:<10} {:<10}".format('Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'))
print("-"*60)  # print separator line
for model, acc, prec, rec, f1 in zip(metrics["Model"], metrics["Accuracy"], metrics["Precision"], metrics["Recall"], metrics["F1 Score"]):
    print("{:<20} {:<10.4f} {:<10.4f} {:<10.4f} {:<10.4f}".format(model, acc, prec, rec, f1))


In [ ]:
model_names = ['Random Forest', 'KNN', 'Naive Bayes', 'Logistic Regression', 'XGBoost']
accuracies = [accuracy_rf, accuracy_knn, accuracy_nb, accuracy_lr, accuracy_xgb] 

# Plotting
fig, ax = plt.subplots(figsize=(9, 7))

# Using consistent colors with a slight alpha transparency and edge color
bars = ax.bar(model_names, accuracies, color='skyblue', alpha=0.75, edgecolor='black')

# Adding data labels with 4 decimal places
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 0.005, "{:.5f}".format(yval), ha='center', va='bottom', fontsize=10)

ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold', labelpad=10)  # labelpad adds some padding to the ylabel
ax.set_xlabel('Model', fontsize=12, fontweight='bold')
ax.set_title('Model Accuracy Comparison', fontsize=16, fontweight='bold')
ax.set_ylim(0, 1.125)  # Added a bit more space at the top for the data labels
ax.set_yticks([i/10 for i in range(11)])
ax.set_yticklabels([i/10 for i in range(11)], fontsize=10)
ax.set_xticks(range(len(model_names)))

# Adjusting font size of the classifier names
ax.set_xticklabels(model_names, rotation=45, ha='right', fontsize=10)  # Reduced font size to 10

ax.grid(axis='y', linestyle='--', linewidth=0.7, alpha=0.6)  # Adding horizontal grid lines

# Adjust the position of the axes to create margins inside the chart
ax.set_position([0.5, 0.5, 0.7, 0.7])  # left, bottom, width, height


plt.show()

In [ ]:
N = float(input("Enter the value of your N: "))
P = float(input("Enter the value of your P: "))
K = float(input("Enter the value of your K: "))
temperature = float(input("Enter the value of your Temperature (C): "))
humidity = float(input("Enter the value of your Humidity: "))
ph = float(input("Enter the value of your Ph: "))
rainfall = float(input("Enter the value of your Rainfall: "))

**Define original feature names and input data**

In [ ]:
original_feature_names = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']
data = np.array([[25, 51, 18, 27.77799528, 54.82130787, 9.45949344, 50.28438729]])
data_df = pd.DataFrame(data, columns=original_feature_names)


**Define Classifier**

In [ ]:
classifiers = {
    "Random Forest": clf_rf,
    "KNN": clf_knn,
    "Naive Bayes": clf_nb,
    "Logistic Regression": clf_lr,
    "XGBoost": clf_xgb,
}

**Ensemble Prediction Function**
<br>We'll encapsulate the ensemble prediction logic into a function.

In [ ]:
def ensemble_predict(classifiers, data_df, le=None):
    # Convert data_df to 2D if it's 1D
    if len(data_df.shape) == 1:
        data_df = data_df.values.reshape(1, -1)
    else:
        data_df = data_df.values
        
    votes = []
    for clf_name, clf in classifiers.items():
        proba = clf.predict_proba(data_df)
        predicted_index = np.argmax(proba)
        
        if clf_name == "XGBoost" and le:
            predicted_label = le.inverse_transform([predicted_index])[0]
        else:
            predicted_label = clf.classes_[predicted_index]

        votes.append(predicted_label)
    
    vote_counts = Counter(votes)
    most_common_votes = vote_counts.most_common()
    if len(most_common_votes) > 1 and most_common_votes[0][1] == most_common_votes[1][1]:
        tied_classes = [vote[0] for vote in most_common_votes if vote[1] == most_common_votes[0][1]]
        return tied_classes[0]
    else:
        return most_common_votes[0][0]


**Ensemble Metrics Calculation**
<br>
This is where we'll genuinely calculate the ensemble metrics by using the hard voting mechanism on a separate test dataset.

In [ ]:
def calculate_ensemble_metrics(classifiers, X_test, y_test, le=None):
    y_preds = [ensemble_predict(classifiers, pd.Series(row), le) for row in X_test.values]
    metrics = {
        'Accuracy': accuracy_score(y_test, y_preds),
        'Precision': precision_score(y_test, y_preds, average='weighted'),
        'Recall': recall_score(y_test, y_preds, average='weighted'),
        'F1 Score': f1_score(y_test, y_preds, average='weighted')
    }
    return metrics


**Visualization and Outputs**
<br>
Here we visualize the ensemble predictions and show the results in a tabular format.

In [ ]:
# Using the ensemble prediction function
ensemble_prediction = ensemble_predict(classifiers, data_df, le)

# Getting metrics using the function (assuming you have X_test and y_test)
ensemble_metrics = calculate_ensemble_metrics(classifiers, X_test, y_test, le)

# Tabular Representation of Model-wise Predictions
print("Model-wise Predictions:")
print("| Model Name            | Prediction   |")
print("|-----------------------|--------------|")

votes = []  # List to collect individual model predictions

for clf_name, clf in classifiers.items():
    proba = clf.predict_proba(data_df.values)
    predicted_index = np.argmax(proba)
    
    if clf_name == "XGBoost" and le:
        predicted_label = le.inverse_transform([predicted_index])[0]
    else:
        predicted_label = clf.classes_[predicted_index]
    print(f"| {clf_name:<21} | {predicted_label:<12} |")
    
    votes.append(predicted_label)  # Add the model's prediction to the votes list

# Visualization: Histogram of Predictions
vote_counts = Counter(votes)
plt.figure(figsize=(10,6))
colors = ['blue', 'green', 'red', 'cyan', 'purple']
plt.bar(vote_counts.keys(), vote_counts.values(), color=colors, alpha=0.75)
plt.yticks(np.arange(0, 6, 1))
plt.ylim(0, 5)
plt.ylabel('Number of Models Voting', fontsize=12)
plt.xlabel('Crops', fontsize=12)
plt.title('Number of Model Votes for Each Predicted Crop', fontsize=14)
plt.xticks(rotation=0, fontsize=11)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.margins(0.1)
plt.tight_layout()
plt.show()

# Tabular Representation of Ensemble Prediction Results
print("\n--------------------------------------------------")
print("              ENSEMBLE PREDICTION RESULT      ")
print("--------------------------------------------------")
print(f"| Crop Predicted   | {ensemble_prediction:<27} |")
print("|------------------|-----------------------------|")
print(f"| Accuracy         | {ensemble_metrics['Accuracy']:<27} |")
print(f"| Precision        | {ensemble_metrics['Precision']:<27} |")
print(f"| Recall           | {ensemble_metrics['Recall']:<27} |")
print(f"| F1 Score         | {ensemble_metrics['F1 Score']:<27} |")
print("--------------------------------------------------")
print("\nNote:")
print("The ensemble prediction is derived from a majority consensus across various models.")
